In [127]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import year,month,hour, when, col, date_format, to_timestamp, round, coalesce
from pyspark.sql import SparkSession
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [3]:
#sc = pyspark.SparkSession('local', 'test')
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

C:\Users\baris\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df = sqlContext.read.option("header",True).option("inferSchema",True) \
     .csv("C:\\Users\\baris\\OneDrive\\desktop\\CSE 6242 Project\\pollution_us_2000_2016.csv")

In [5]:
df=df.withColumn("year",year("Date Local"))

In [6]:
df1=df.select("State","County","year","NO2 Mean","CO Mean",'O3 Mean','SO2 Mean')\
.groupBy("State","County","year").avg("NO2 Mean","CO Mean",'O3 Mean','SO2 Mean')

In [7]:
df1p=df1.toPandas()

In [12]:
df2p=pd.read_csv("C:\\Users\\baris\\OneDrive\\desktop\\CSE 6242 Project\death_data.csv")

In [13]:
us_state_abbrev={'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA': 'California','CO':'Colorado',\
                                 'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','IA':'Iowa',\
                                 'ID':'Idaho','IL': 'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana', \
                                'MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri', \
                                'NC':'North Carolina','ND':'North Dakota','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico', \
                                'NV':'Nevada','NY':'New York','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania' ,\
                                'RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas', \
                                'UT':'Utah','VA':'Virginia','WA':'Washington','WI':'Wisconsin','WY':'Wyoming'}

In [14]:
df2p['State']=df2p['State'].map(us_state_abbrev)

In [15]:
df_all=df1p.sort_values(by=['year'])

In [16]:
df3p=pd.merge(df_all,df2p,how="inner", left_on=['State','County','year'], right_on=['State','County_Name','Year'])

In [17]:
df3p=df3p.T.drop_duplicates().T

In [27]:
df4p=pd.read_csv("C:\\Users\\baris\\OneDrive\\desktop\\CSE 6242 Project\\manufaturing_totals_2000_to_2016_restructure.csv")

In [28]:
df5p=df4p.dropna()

In [29]:
#df5p=df4p.melt(id_vars=["county", "state"], 
#        var_name="year", 
#        value_name="number of factories")
df5p

,county,state,year,fat_amount
0,Los Angeles,CA,2000,35154
1,Cook,IL,2000,14998
2,Orange,CA,2000,11332
3,Harris,TX,2000,8502
4,New York,NY,2000,8382
...,...,...,...,...
53222,Mcmullen,TX,2016,5
53223,Leslie,KY,2016,5
53224,Lyman,SD,2016,5
53225,Loup,NE,2016,5


In [30]:
df5p['state']=df5p['state'].map(us_state_abbrev)

In [31]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [32]:
df5p = swap_columns(df5p, 'county', 'state')
df5p=df5p.rename(columns={"state": "State", "county": "County", "fat_amount": "num_factory"})

In [33]:
df5p

,State,County,year,num_factory
0,California,Los Angeles,2000,35154
1,Illinois,Cook,2000,14998
2,California,Orange,2000,11332
3,Texas,Harris,2000,8502
4,New York,New York,2000,8382
...,...,...,...,...
53222,Texas,Mcmullen,2016,5
53223,Kentucky,Leslie,2016,5
53224,South Dakota,Lyman,2016,5
53225,NaN,Loup,2016,5


In [45]:
df6p=pd.merge(df3p,df5p,how="inner", on=['State','County','year'])

In [46]:
df6p

,State,County,year,avg(NO2 Mean),avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),Population,Area,Deaths,...,canc_death,resp_circ_death,Crude Rate,pollution_Crude_Rate,resp_Crude_Rate,circ_Crude_Rate,canc_Crude_Rate,resp_circ_Crude_Rate,population_density,num_factory
0,North Carolina,Mecklenburg,2000,15.374838,0.623766,0.03522,3.772487,695454,549.74,4505,...,1020,1981,647.778286,431.516678,56.509848,228.340048,146.666782,284.849897,1265.059846,1960
1,Kansas,Wyandotte,2000,16.518028,0.984364,0.025053,1.378182,157882,155.7,1314,...,288,585,832.267136,552.944604,97.541202,272.988688,182.414715,370.529889,1014.01413,494
2,Oklahoma,Cherokee,2000,3.67854,0.041166,0.034708,0.198155,42521,776.45,228,...,37,130,536.20564,392.747113,35.276687,270.454599,87.015827,305.731286,54.763346,38
3,Michigan,Wayne,2000,22.640061,0.277001,0.02335,5.415251,2061162,672.26,20218,...,4435,10044,980.903005,702.467831,74.957718,412.340224,215.16989,487.297942,3066.0191,4336
4,Pennsylvania,Lancaster,2000,11.187757,0.23918,0.026628,2.762724,470658,983.9,4137,...,952,2082,878.982191,644.629434,81.375436,360.983984,202.270013,442.35942,478.359589,1898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,Alabama,Jefferson,2016,9.524873,0.209514,0.028348,0.835118,659521,1123.87,7014,...,1444,2610,1063.499115,614.688539,92.946244,302.795514,218.946781,395.741758,586.830327,2810
1090,New York,Queens,2016,17.874275,0.257096,0.02405,0.836166,2333054,177.35,14393,...,3537,7115,616.916711,456.568944,51.991938,252.973142,151.603863,304.96508,13155.083169,6195
1091,California,Solano,2016,8.40495,0.42257,0.023891,0.611533,440207,906.85,3148,...,848,1193,715.118115,463.645512,66.786762,204.222105,192.636646,271.008866,485.424271,1240
1092,Colorado,Denver,2016,25.17045,0.437617,0.02121,1.140453,693060,154.88,4505,...,1010,1637,650.015872,381.929414,64.929443,171.269443,145.730528,236.198886,4474.819215,4185


In [47]:
df7p=df6p[['avg(NO2 Mean)','avg(CO Mean)','avg(O3 Mean)','avg(SO2 Mean)', 'population_density','num_factory']]

In [40]:
cars=pd.read_csv("C:\\Users\\baris\\OneDrive\\desktop\\CSE 6242 Project\\CA_county_cars.csv")

In [48]:
df7p_cars=pd.merge(df6p,cars,how="inner", on=['State','County','year'])

In [51]:
df8p=df7p_cars[['avg(NO2 Mean)','avg(CO Mean)','avg(O3 Mean)','avg(SO2 Mean)', 'population_density','num_factory','Autos','Trucks','Total Vehicles']]

In [65]:
df8p = df8p.astype(float)
df7p = df7p.astype(float)

In [66]:
#df7p.to_csv('df7.csv')
df7p.corr()

,avg(NO2 Mean),avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),population_density,num_factory
avg(NO2 Mean),1.000000,0.587601,-0.525557,0.416436,0.437043,0.294672
avg(CO Mean),0.587601,1.000000,-0.392298,0.149995,0.186131,0.179794
avg(O3 Mean),-0.525557,-0.392298,1.000000,-0.170427,-0.336845,-0.160823
avg(SO2 Mean),0.416436,0.149995,-0.170427,1.000000,0.275931,-0.114577
population_density,0.437043,0.186131,-0.336845,0.275931,1.000000,0.047009
num_factory,0.294672,0.179794,-0.160823,-0.114577,0.047009,1.000000


In [64]:
df8p.corr()

,avg(NO2 Mean),avg(CO Mean),avg(O3 Mean),avg(SO2 Mean),population_density,num_factory,Autos,Trucks,Total Vehicles
avg(NO2 Mean),1.000000,0.511820,-0.105867,0.166962,0.257804,0.469287,0.594719,0.656627,0.613328
avg(CO Mean),0.511820,1.000000,-0.288117,0.128114,-0.044839,0.102574,0.145071,0.141606,0.143774
avg(O3 Mean),-0.105867,-0.288117,1.000000,-0.003403,-0.359067,-0.080305,-0.118654,-0.018385,-0.095352
avg(SO2 Mean),0.166962,0.128114,-0.003403,1.000000,0.019531,-0.139716,-0.004078,0.042580,0.006573
population_density,0.257804,-0.044839,-0.359067,0.019531,1.000000,0.502800,0.525918,0.437381,0.506228
num_factory,0.469287,0.102574,-0.080305,-0.139716,0.502800,1.000000,0.905554,0.848553,0.894844
Autos,0.594719,0.145071,-0.118654,-0.004078,0.525918,0.905554,1.000000,0.978361,0.998608
Trucks,0.656627,0.141606,-0.018385,0.042580,0.437381,0.848553,0.978361,1.000000,0.987782
Total Vehicles,0.613328,0.143774,-0.095352,0.006573,0.506228,0.894844,0.998608,0.987782,1.000000


In [71]:
train7,test7 = train_test_split(df7p, test_size=0.2, random_state=42)

In [72]:
X7_train = train7.drop(['avg(NO2 Mean)'], axis=1)
y7_train = train7['avg(NO2 Mean)']
X7_test = test7.drop(['avg(NO2 Mean)'], axis=1)
y7_test = test7['avg(NO2 Mean)']

In [105]:
p_model = LinearRegression()
p_model.fit(X7_train, y7_train)

predicted_no2 = p_model.predict(X7_test)

mean_squared_error(y7_test, predicted_no2)


11.79445004504477

In [79]:
mean_absolute_error(y7_test, predicted_no2)

2.639403969164844

In [115]:
p_model.score(X7_test,y7_test)

0.6542914724927071

In [103]:
vif_data = pd.DataFrame() 
vif_data["feature"] = df7p.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df7p.values, i) 
                          for i in range(len(df7p.columns))] 

vif_data

In [118]:
p_model1 = DecisionTreeRegressor(max_depth=5,random_state=7)
p_model1.fit(X7_train, y7_train)

predicted_no2 = p_model1.predict(X7_test)

mean_squared_error(y7_test, predicted_no2)

11.9876537155538

In [119]:
mean_absolute_error(y7_test, predicted_no2)

2.6962862943253434

In [134]:
p_model11 = RandomForestRegressor(n_estimators=100,
                                  random_state=7)

p_model11.fit(X7_train, y7_train)

predicted_no22 = p_model11.predict(X7_test)

mean_squared_error(y7_test, predicted_no22)

6.228282772477633

In [135]:
mean_absolute_error(y7_test, predicted_no22)

1.868858897992442

In [116]:
train8,test8 = train_test_split(df8p, test_size=0.2, random_state=42)

In [117]:
X8_train = train8.drop(['avg(NO2 Mean)'], axis=1)
y8_train = train8['avg(NO2 Mean)']
X8_test = test8.drop(['avg(NO2 Mean)'], axis=1)
y8_test = test8['avg(NO2 Mean)']

In [122]:
p_model2 = LinearRegression()
p_model2.fit(X8_train, y8_train)

predicted_no22 = p_model2.predict(X8_test)

mean_squared_error(y8_test, predicted_no22)

8.284689383518991

In [126]:
mean_absolute_error(y8_test, predicted_no22)

2.347262469146624

In [124]:
p_model2.score(X8_test,y8_test)

0.726482192118924